### Building a Wine Terroir Dataset

In this notebook, we will pull in data from various sources to build a dataset with information on wine terroir (altitude, weather and soil type). 

In [1]:
import pandas as pd
import numpy as np
import googlemaps
import os
import datetime
from dateutil.relativedelta import relativedelta
import requests 

#### 1. The Raw Wine Review Dataset

Take the file for each grape variety and concatenate into one dataset with all our wines.

In [5]:
file_repository = 'C:/Users/roald/Documents/data_science_projects/wine_data'
all_dfs = []

for f in os.listdir(file_repository):
    file_loc = file_repository + '/' + f
    df = pd.read_csv(file_loc, index_col=0)
    all_dfs.append(df)

all_dfs = pd.concat(all_dfs, axis=0, ignore_index=True)
all_dfs.head()

,Alcohol,Appellation,Bottle Size,Category,Country,Date Published,Description,Designation,Importer,Name,...,Province,Rating,Region,Reviewer,Reviewer Twitter Handle,Subregion,User Avg Rating,Variety,Vintage,Winery
0,14.9%,"Napa Valley, Napa, California, US",750 ml,Red,US,7/1/2019,Made in partnership with Alpha Omega Winery's ...,MR,NaN,Michel Rolland Napa Valley 2014 MR Red (Napa V...,...,California,95.0,Napa,Virginie Boone,@vboone,Napa Valley,Not rated yet [Add Your Review],Bordeaux-style Red Blend,2014.0,Michel Rolland Napa Valley
1,14.5%,"Columbia Valley (WA), Columbia Valley, Washing...",750 ml,Red,US,7/1/2019,Cabernet Sauvignon from esteemed Cold Creek Vi...,NaN,NaN,21 Grams 2014 Red (Columbia Valley (WA)),...,Washington,94.0,Columbia Valley,Sean P. Sullivan,@wawinereport,Columbia Valley (WA),Not rated yet [Add Your Review],Bordeaux-style Red Blend,2014.0,21 Grams
2,13.6%,"Paso Robles, Central Coast, California, US",750 ml,Red,US,7/1/2019,Lush black-raspberry and black-plum aromas com...,Nikiara,NaN,Le Vigne 2016 Nikiara Red (Paso Robles),...,California,92.0,Central Coast,Matt Kettmann,@mattkettmann,Paso Robles,Not rated yet [Add Your Review],Bordeaux-style Red Blend,2016.0,Le Vigne
3,14.8%,"Adelaida District, Central Coast, California, US",750 ml,Red,US,7/1/2019,"Bright aromas of blackberry, violet and cedar ...",NaN,NaN,Brecon Estate 2016 Meritage (Adelaida District),...,California,92.0,Central Coast,Matt Kettmann,@mattkettmann,Adelaida District,Not rated yet [Add Your Review],"Meritage, Bordeaux-style Red Blend",2016.0,Brecon Estate
4,14.5%,"Aconcagua Valley, Chile",750 ml,Red,Chile,7/1/2019,Spicy cherry and coffee aromas drive a traditi...,Capa,WTWM Imports,Conde de Velázquez 2014 Capa Red (Aconcagua Va...,...,Aconcagua Valley,91.0,NaN,Michael Schachner,@wineschach,NaN,Not rated yet [Add Your Review],Bordeaux-style Red Blend,2014.0,Conde de Velázquez


Drop any observations we won't be able to use (wines that are not red/white/rose, and any observations without a description or an appellation that can tell us where it is from).

In [6]:
only_usable_categories = all_dfs.loc[all_dfs['Category'].isin(['Red', 'White', 'Rose'])]
only_usable_wines = only_usable_categories.dropna(subset=['Appellation', 'Description', 'Vintage'])
unique_usable_wines = only_usable_wines.drop_duplicates()
unique_usable_wines.shape

(139599, 21)

#### 2. Geocoding

For every appellation in our dataset, let's return the latitude and the longitude.

In [27]:
gmaps_key = googlemaps.Client(key=ENTER_YOUR_API_KEY_HERE)

all_addresses = list(set(unique_usable_wines['Appellation']))

address_coordinates = dict()
for address in all_addresses:
    geocode_result = gmaps_key.geocode(address)
    try:
        lat = geocode_result[0]['geometry']['location']['lat']
        lon = geocode_result[0]['geometry']['location']['lng']
    except:
        lat = None
        lon = None
    address_coordinates[address] = (lat, lon)

address_coordinates_df = pd.DataFrame.from_dict(address_coordinates, orient='index', columns=['Latitude', 'Longitude'])
address_coordinates_df.to_csv('address_coordinates.csv')

Now that we have tried to retrieve the coordinates of all the various appellations, we can eliminate those wines for which we did not successfully find a longitude and latitude.

In [7]:
address_coordinates_df = pd.read_csv('address_coordinates.csv', index_col=0)
address_coordinates_df_nonulls = address_coordinates_df.dropna()
acceptable_appellations = list(address_coordinates_df_nonulls.index)

only_usable_addresses = unique_usable_wines.loc[unique_usable_wines['Appellation'].isin(acceptable_appellations)]
print(only_usable_addresses.shape)

(138553, 21)


#### 3. Weather Data

Thankfully, only a small portion of our dataset does not have a useful geotag. Next, we can turn our attention to collecting weather data. To this effect, we will use the GHCN weather dataset available through Google's BigQuery.

In [8]:
import datalab.bigquery as bq
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="C:/Users/roald/Documents/data_science_projects/Exploring Terroir/wine-data-science-f2f741cae037.json"
# noaa = bq_helper.BigQueryHelper(active_project="bigquery-public-data", dataset_name="ghcn_d")
# bq_assistant = BigQueryHelper("bigquery-public-data", "ghcn_d")

As a first step, let's take a look at which addresses have usable weather data. To do this, we will find the closest weather station to each set of coordinates. 

In [157]:
closest_weather_station = """
SELECT
  id,
  DEGREES(ACOS(SIN(RADIANS(latitude)) * SIN(RADIANS($lat)) + COS(RADIANS(latitude)) * COS(RADIANS($lat)) * COS(RADIANS(longitude - $lon)))) * 60 * 1.515 * 1.609344 AS dist_kms
FROM
  [bigquery-public-data:ghcn_d.ghcnd_stations]
ORDER BY dist_kms ASC
LIMIT 1
"""

In [158]:
locations_weather_stations = []
for index, row in address_coordinates_df.iterrows():
    try:
        lati = row['Latitude']
        long = row['Longitude']
        closest_station = bq.Query(closest_weather_station, lat=lati, lon=long).to_dataframe()
        station_id = closest_station['id'][0]
        station_distance = closest_station['dist_kms'][0]
        locations_weather_stations.append([index, lati, long, station_id, station_distance])
    except:
        continue

locations_weather_stations = pd.DataFrame(locations_weather_stations, columns=['Appellation', 'Latitude', 'Longitude', 'Station_ID', 'Station_Distance_KMs'])
locations_weather_stations.to_csv('closest_weather_stations.csv')
locations_weather_stations.head()

,Appellation,Latitude,Longitude,Station_ID,Station_Distance_KMs
0,"Pla de Bages, Catalonia, Spain",41.720580,1.839796,SPE00156540,10.415249
1,"Terasele Dunarii, Romania",44.622176,22.651475,ROE00100903,2.492714
2,"California, California Other, California, US",35.125801,-117.985904,US1CAKN0002,4.838883
3,"Châteaumeillant, Loire Valley, France",46.562103,2.201875,FRW00034048,64.363272
4,"Polkadraai Hills, South Africa",-33.960845,18.800415,SF000216550,9.574565


We can now eliminate any wines that do not have a weather station within reasonable distance (for the purposes of this analysis, we have picked 150km as an arbitrary cutoff point).

In [8]:
locations_weather_stations = pd.read_csv('closest_weather_stations.csv', index_col=0)
locations_weather_stations.sort_values(by='Station_Distance_KMs', ascending=False, inplace=True)

locations_weather_close = locations_weather_stations.loc[locations_weather_stations['Station_Distance_KMs'] < 150]
locations_weather_close.head(10)

,Appellation,Latitude,Longitude,Station_ID,Station_Distance_KMs
1057,"Rose Valley, Bulgaria",42.667181,25.409197,BUM00015730,148.829881
386,"Stirling, New Zealand",-46.247506,169.780235,NZ000093844,148.269769
398,"Lemnos, Greece",39.919841,25.141484,TUM00017112,147.491912
548,"Mount Athos, Greece",40.264493,24.215273,GRM00016622,143.583165
291,"Marcillac, Southwest France, France",45.269046,-0.523209,FR000007510,140.967197
293,"Rapel Valley, Chile",-33.942936,-71.736178,CIM00085574,140.907690
538,"Rapel Valley-Casablanca Valley, Chile",-33.942936,-71.736178,CIM00085574,140.907690
586,"Altos de Mendoza, Mendoza Province, Argentina",-32.653179,-70.010868,CIM00085577,140.738681
1367,"Valle de Uco, Mendoza Province, Argentina",-33.772625,-69.156521,ARM00087416,135.386344
1281,"Alto Valle de Uco, Mendoza Province, Argentina",-33.772625,-69.156521,ARM00087416,135.386344


In [9]:
weather_available = only_usable_addresses.loc[only_usable_addresses['Appellation'].isin(list(locations_weather_close['Appellation']))]
vintage_appellations = weather_available[['Appellation', 'Vintage']].drop_duplicates().sort_values(by=['Appellation', 'Vintage'])

vintage_appellations = pd.merge(vintage_appellations, locations_weather_close, left_on='Appellation', right_on='Appellation')
vintage_appellations['northern_hemisphere'] = [0 if x < 0 else 1 for x in vintage_appellations['Latitude']]
vintage_appellations.head()

,Appellation,Vintage,Latitude,Longitude,Station_ID,Station_Distance_KMs,northern_hemisphere
0,"Abruzzo, Central Italy, Italy",1998.0,42.192012,13.728917,ITM00016219,90.143885,1
1,"Abruzzo, Central Italy, Italy",2014.0,42.192012,13.728917,ITM00016219,90.143885,1
2,"Abruzzo, Central Italy, Italy",2016.0,42.192012,13.728917,ITM00016219,90.143885,1
3,"Abruzzo, Central Italy, Italy",2017.0,42.192012,13.728917,ITM00016219,90.143885,1
4,"Achaia, Greece",2005.0,38.115873,21.952249,GRE00155860,98.725446,1


To retrieve actual weather data, we need to stipulate the date parameters for which we want to pull various pieces of data. We want to note weather conditions specific to the year in which each wine was produced, and align this roughly with the production cycle of the wine. For most of the northern hemisphere, spring begins in March, while this is September for much of the southern hemisphere. We retrieve monthly data for each of the spring, summer and autumn months. 

In [19]:
def generate_month_description(season, month_number, vintage, hemisphere):
    
    if hemisphere == 1:
        if season == 'spring':
            month = month_number + 2
        elif season == 'summer':
            month = month_number + 4
        else:
            month = month_number + 6
    elif hemisphere == 0:
        if season == 'spring':
            month = month_number + 8
        elif season == 'summer':
            if month_number == 1:
                month = 9
            else:
                month = month_number - 1
        else:
            month = month_number + 2
    
    if hemisphere ==0 and (season == 'spring' or (season == 'summer' and month_number == 1)):
        year = int(vintage) - 1
    else: 
        year = int(vintage)
    
    date_string = str(year) + '-' + str(month) + '-1'
    date_time_obj = datetime.datetime.strptime(date_string, '%Y-%m-%d')
    
    return date_time_obj

for s in ['spring', 'summer', 'autumn']:
    for i in range(1, 4):
        column_name = str(s) + '_month_' + str(i)
        vintage_appellations[column_name] = vintage_appellations.apply(lambda x: generate_month_description(s, i, x['Vintage'], x['northern_hemisphere']), axis=1)

appellations_dates = pd.melt(vintage_appellations, id_vars=['Appellation', 'Station_ID', 'Vintage'], value_vars=['spring_month_1', 'spring_month_2', 'spring_month_3', 
                                                                                                      'summer_month_1', 'summer_month_2', 'summer_month_3', 
                                                                                                      'autumn_month_1', 'autumn_month_2', 'autumn_month_3'])
appellations_dates.head(50)

,Appellation,Station_ID,Vintage,variable,value
0,"Abruzzo, Central Italy, Italy",ITM00016219,1998.0,spring_month_1,1998-03-01
1,"Abruzzo, Central Italy, Italy",ITM00016219,2014.0,spring_month_1,2014-03-01
2,"Abruzzo, Central Italy, Italy",ITM00016219,2016.0,spring_month_1,2016-03-01
3,"Abruzzo, Central Italy, Italy",ITM00016219,2017.0,spring_month_1,2017-03-01
4,"Achaia, Greece",GRE00155860,2005.0,spring_month_1,2005-03-01
5,"Achaia, Greece",GRE00155860,2007.0,spring_month_1,2007-03-01
6,"Achaia, Greece",GRE00155860,2008.0,spring_month_1,2008-03-01
7,"Achaia, Greece",GRE00155860,2014.0,spring_month_1,2014-03-01
8,"Aconcagua Costa, Chile",CI000085543,2013.0,spring_month_1,2012-09-01
9,"Aconcagua Costa, Chile",CI000085543,2014.0,spring_month_1,2013-09-01


In [13]:
appellations_dates.shape

(95877, 5)

In [62]:
samp_quer1 = """
SELECT
  id,
  calendar_year,
  calendar_month,
  AVG(prcp) as prcp,
  MIN(tmin) as tmin,
  MAX(tmax) as tmax,
  AVG(tmin) as avg_tmin,
  AVG(tmax) as avg_tmax
FROM (
    SELECT
        wx.id,
        wx.date,
        IF (wx.element = 'PRCP', wx.value/10, NULL) AS prcp,
        IF (wx.element = 'TMIN', wx.value/10, NULL) AS tmin,
        IF (wx.element = 'TMAX', wx.value/10, NULL) AS tmax,
        YEAR(wx.date) as calendar_year,
        MONTH(wx.date) as calendar_month
    FROM
        [bigquery-public-data:ghcn_d.ghcnd_$year] as wx
    WHERE 
        id in $weather_stations
)
GROUP BY
  id, calendar_month, calendar_year;
"""

def retrieve_weather_info(year, weather_stations):
    weather_info = bq.Query(samp_quer1, year=year, weather_stations=weather_stations).to_dataframe()
    return weather_info

In [66]:
appellations_dates_unique = appellations_dates[['Station_ID', 'Vintage']].drop_duplicates()

all_vintages = list(set(appellations_dates_unique['Vintage']))

weather_info_df = pd.DataFrame(columns=['id', 'calendar_year', 'calendar_month', 'prcp', 'tmin', 'tmax', 'avg_tmin', 'avg_tmax'])

for v in all_vintages:
    appellations_dates_vintage = appellations_dates_unique.loc[appellations_dates_unique['Vintage']==v]
    appellation_ids = tuple(appellations_dates_vintage['Station_ID'])
    weather_info = retrieve_weather_info(int(v), appellation_ids)
    weather_info_df = weather_info_df.append(weather_info, ignore_index=True)

weather_info_df.to_csv('all_weather_df.csv')
print(weather_info_df.head())

            id calendar_year calendar_month  prcp  tmin  tmax   avg_tmin  \
0  FRE00104949          1976              7   NaN   7.8  36.2  15.041935   
1  FRE00104949          1976              9   NaN   4.7  25.5   9.473333   
2  FRE00104949          1976              2   NaN  -7.4  15.4  -0.444828   
3  FRE00104949          1976             11   NaN  -3.6  13.3   3.676667   
4  FRE00104949          1976              3   NaN  -7.6  17.4  -0.377419   

    avg_tmax  
0  27.638710  
1  19.486667  
2   5.975862  
3   8.486667  
4   9.416129  


In [15]:
weather_info_df = pd.read_csv('all_weather_df.csv', index_col=0)

def build_date(year, month):
    date_string = str(year) + '-' + str(month) + '-1'
    date_time_obj = datetime.datetime.strptime(date_string, '%Y-%m-%d')
    return date_time_obj

weather_info_df['date'] = weather_info_df.apply(lambda x: build_date(x['calendar_year'], x['calendar_month']), axis=1)
weather_info_df.head()    

,id,calendar_year,calendar_month,prcp,tmin,tmax,avg_tmin,avg_tmax,date
0,FRE00104949,1976,7,NaN,7.8,36.2,15.041935,27.638710,1976-07-01
1,FRE00104949,1976,9,NaN,4.7,25.5,9.473333,19.486667,1976-09-01
2,FRE00104949,1976,2,NaN,-7.4,15.4,-0.444828,5.975862,1976-02-01
3,FRE00104949,1976,11,NaN,-3.6,13.3,3.676667,8.486667,1976-11-01
4,FRE00104949,1976,3,NaN,-7.6,17.4,-0.377419,9.416129,1976-03-01


In [20]:
wine_weather_data = pd.merge(appellations_dates, weather_info_df, left_on=['Station_ID', 'value'], right_on=['id', 'date'])
wine_weather_data = wine_weather_data[['Appellation', 'Vintage', 'variable', 'prcp', 'tmin', 'tmax', 'avg_tmin', 'avg_tmax']]
wine_weather_data.set_index(['Appellation', 'Vintage', 'variable'], inplace=True)
wine_weather_data_pivot = wine_weather_data.unstack(level=-1)
# df.columns = df.columns.map('_'.join)
wine_weather_data_pivot.columns = ['_'.join((col[1], col[0])) for col in wine_weather_data_pivot.columns]
wine_weather_data_pivot.reset_index(inplace=True)
wine_weather_data_pivot.head()
wine_weather_data_pivot.to_csv('weather_data.csv')

#### 4. Soil Data

Next, we will retrieve data on the attributes of the soil where our various wines are grown. We can make use of the soilgrids API to do this. We will identify a broad range of soil attributes to investigate at this stage. Feeding the API the coordinates for each appellation will allow us to paint a profile for the soil in the general vicinity where a wine is produced. This will not be able to account for very local variations in the chemical composition of soil, but will allow us to pick up on general geographical differences in the structure of soil types. We will retrieve this data for a variety of soil depths (0cm, 30cm and 100cm).

In [12]:
def get_soil_info(appel, lat, lon):
    
    def grab_specific_soil_info(json_resp, soil_code):
        try:
            var1, var2, var3 = [v for k, v in json_resp['properties'][soil_code]['M'].items()]
            return var1, var2, var3
        except:
            return np.nan, np.nan, np.nan
    
    relevant_soil_codes = ['AWCh1', 'AWCh2', 'AWCh3', 'BLDFIE', 'CECSOL', 'CLYPPT', 'ORCDRC', 'PHIHOX',
                           'SLTPPT', 'SNDPPT', 'EXBX', 'ENAX', 'EMGX', 'EXKX', 'ECAX', 'EACKCL', 'ALUM3S', 'CRFVOL', 'NTO']
    
    resp = requests.get("https://rest.soilgrids.org/query?lon=" 
                        + str(lon) + "&lat=" + str(lat) 
                        + "&attributes=EXBX,ENAX,EMGX,EXKX,ECAX,EACKCL,ALUM3S,CRFVOL,SNDPPT,SLTPPT,CLYPPT,ORCDRC,BLD,CEC,PHIHOX,h1,h2,h3,pwp,PTF.coef,TAXGWRBMajor,NTO&depths=sl1,sl4,sl6") 
    json_resp = resp.json()
    
    all_globalvars = []
    
    for code in relevant_soil_codes:
        varnames = [str(code) + '_0', str(code) + '_30', str(code) + '_100']
        globals()[varnames[0]], globals()[varnames[1]], globals()[varnames[2]] = grab_specific_soil_info(json_resp, code)
        all_globalvars.extend([globals()[varnames[0]], globals()[varnames[1]], globals()[varnames[2]]])
    
    soilmask = json_resp['properties']['soilmask']
    all_globalvars.append(soilmask)
    all_globalvars.append(appel)
    
    return all_globalvars

In [14]:
appellations_coordinates = vintage_appellations[['Appellation', 'Latitude', 'Longitude']].drop_duplicates()

soil_infos = []
for index, row in appellations_coordinates.iterrows():
    appel = row['Appellation']
    lat = row['Latitude']
    lon = row['Longitude']
    soil_info = get_soil_info(appel, lat, lon)
    soil_infos.append(soil_info)

soil_df_colnames = []
relevant_soil_codes = ['AWCh1', 'AWCh2', 'AWCh3', 'BLDFIE', 'CECSOL', 'CLYPPT', 'ORCDRC', 'PHIHOX',
                        'SLTPPT', 'SNDPPT', 'EXBX', 'ENAX', 'EMGX', 'EXKX', 'ECAX', 'EACKCL', 'ALUM3S', 'CRFVOL', 'NTO']
for s in relevant_soil_codes:
    varnames = [str(s) + '_0', str(s) + '_30', str(s) + '_100']
    soil_df_colnames.extend(varnames)
soil_df_colnames.append('soilmask')
soil_df_colnames.append('appellation')

soil_infos_df = pd.DataFrame(soil_infos, columns=soil_df_colnames)
soil_infos_df.to_csv('soil_data.csv')
soil_infos_df.head()

,AWCh1_0,AWCh1_30,AWCh1_100,AWCh2_0,AWCh2_30,AWCh2_100,AWCh3_0,AWCh3_30,AWCh3_100,BLDFIE_0,...,ALUM3S_30,ALUM3S_100,CRFVOL_0,CRFVOL_30,CRFVOL_100,NTO_0,NTO_30,NTO_100,soilmask,appellation
0,15.0,14.0,11.0,12.0,11.0,9.0,10.0,9.0,7.0,1264.0,...,NaN,NaN,17.0,18.0,18.0,NaN,NaN,NaN,soil,"Abruzzo, Central Italy, Italy"
1,16.0,12.0,10.0,13.0,9.0,8.0,11.0,8.0,7.0,1284.0,...,NaN,NaN,21.0,20.0,21.0,NaN,NaN,NaN,soil,"Achaia, Greece"
2,13.0,13.0,12.0,10.0,10.0,9.0,8.0,8.0,7.0,1458.0,...,NaN,NaN,15.0,27.0,20.0,NaN,NaN,NaN,soil,"Aconcagua Costa, Chile"
3,13.0,13.0,12.0,10.0,10.0,9.0,8.0,8.0,7.0,1458.0,...,NaN,NaN,15.0,27.0,20.0,NaN,NaN,NaN,soil,"Aconcagua Valley, Chile"
4,11.0,10.0,9.0,9.0,8.0,7.0,7.0,7.0,6.0,1565.0,...,NaN,NaN,8.0,8.0,8.0,NaN,NaN,NaN,soil,"Adelaida District, Central Coast, California, US"


#### 5. Altitude

Finally, we will collect data on the altitudes at which our various wines are produced. Again, we we will use the geographical coordinates in order to grab this information. In keeping with how we obtained data on geographical coordinates and weather, we will again use a Google API to retrieve altitude information. The Elevation API can help us here, requiring only a series of latitudes, longitudes and an API key.

In [17]:
import urllib
import simplejson

altitude_infos = []
appellations = []

for index, row in appellations_coordinates.iterrows():
    lat = row['Latitude']
    lon = row['Longitude']
    appellation = row['Appellation']
    latlon = str(lat) + ',' + str(lon)
    altitude_infos.append(latlon)
    appellations.append(appellation)

# the API cannot handle all requests at once - we will make chunks of 500 to address this issue
def divide_chunks(l, n):   
    for i in range(0, len(l), n):  
        yield l[i:i + n]

all_altitude_chunks = divide_chunks(altitude_infos, 500)
appellation_chunks = divide_chunks(appellations, 500)

lat_lon_elevation = []
for a, b in zip(all_altitude_chunks, appellation_chunks):
    all_altitude_info = '|'.join(a)

    api_string = 'https://maps.googleapis.com/maps/api/elevation/json?locations=' + str(all_altitude_info) + ENTER_YOUR_API_KEY_HERE
    resp = requests.get(api_string)
    json_resp = resp.json()
    
    i = 0
    for e in json_resp['results']:
        elevation = e['elevation']
        lat = e['location']['lat']
        lon = e['location']['lng']
        appel = b[i]
        i += 1
        lat_lon_elevation.append([appel, lat, lon, elevation])

elevation_df = pd.DataFrame(lat_lon_elevation, columns=['Appellation', 'lat', 'lon', 'elevation'])
elevation_df.to_csv('elevation_data.csv')
elevation_df.head()

,Appellation,lat,lon,elevation
0,"Abruzzo, Central Italy, Italy",42.192012,13.728917,725.039124
1,"Achaia, Greece",38.115873,21.952249,981.088806
2,"Aconcagua Costa, Chile",-32.803550,-70.944224,941.682434
3,"Aconcagua Valley, Chile",-32.803550,-70.944224,941.682434
4,"Adelaida District, Central Coast, California, US",35.645397,-120.874138,284.341766
